#AlexNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_height, img_width = 28, 28
num_classes = 10

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


x_train = np.stack([x_train, x_train, x_train], axis=-1)
x_test = np.stack([x_test, x_test, x_test], axis=-1)
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(
    x_train, y_train_categorical,
    epochs=10,
    batch_size=64,
    validation_data=(x_test, y_test_categorical)
)


y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_test

print("Classification Report:")
print(classification_report(y_true, y_pred_classes))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))

report = classification_report(y_true, y_pred_classes, output_dict=True)
accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1_score}")

model.save('alexnet_mnist_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.8782 - loss: 0.3776 - val_accuracy: 0.9842 - val_loss: 0.0482
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9838 - loss: 0.0536 - val_accuracy: 0.9907 - val_loss: 0.0310
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - accuracy: 0.9879 - loss: 0.0403 - val_accuracy: 0.9900 - val_loss: 0.0346
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9908 - loss: 0.0298 - val_accuracy: 0.9897 - val_loss: 0.0315
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9922 - loss: 0.0240 - val_accuracy: 0.9913 - val_loss: 0.0283
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9940 - loss: 0.0187 - val_accuracy: 0.9917 - val_loss: 0.0267
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9956 - loss: 0.0156 - val_accuracy: 0.9903 - val_loss: 0.0380
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9955 - loss: 0.0142 - val_accurac

Accuracy: 0.992
Precision: 0.9919787073688081
Recall: 0.9919026297331783
F1-score: 0.9919302131878374


#VGG16

In [ ]:
!pip install tensorflow scikit-learn numpy
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_height, img_width = 32, 32
num_classes = 10
x_train_resized = tf.image.resize(x_train[..., np.newaxis], (img_height, img_width))
x_train_resized = tf.repeat(x_train_resized, 3, axis=-1)
x_test_resized = tf.image.resize(x_test[..., np.newaxis], (img_height, img_width))
x_test_resized = tf.repeat(x_test_resized, 3, axis=-1)
x_train_resized = x_train_resized.numpy() / 255.0
x_test_resized = x_test_resized.numpy() / 255.0
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(
    x_train_resized, y_train_categorical,
    epochs=2,
    batch_size=128,
    validation_data=(x_test_resized, y_test_categorical)
)
y_pred = model.predict(x_test_resized)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_test
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))
report = classification_report(y_true, y_pred_classes, output_dict=True)
accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
model.save('vgg16_mnist_model.h5')

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 64s 110ms/step - accuracy: 0.9286 - loss: 0.2332 - val_accuracy: 0.9773 - val_loss: 0.0769
Epoch 2/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 58s 77ms/step - accuracy: 0.9888 - loss: 0.0391 - val_accuracy: 0.9937 - val_loss: 0.0233
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       1.00      1.00      1.00      1135
           2       1.00      0.99      0.99      1032
           3       0.99      1.00      1.00      1010
           4       0.99      1.00      1.00       982
           5       1.00      0.98      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      1.00      0.99      1028
           8       0.99      1.00      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

Confusion Matrix:
[[ 978    0    0    0    0    0    0    1    1    0]
 [   0 1131    0    0    0    0    3    1    0    0]
 [   0    0 1020    1    2    0    0    6    3    0]
 [   0    0    1 1006    0    1    0    1

#ResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

(x_train, y_train), (x_test, y_test) = mnist.load_data()
img_height, img_width = 28, 28
num_classes = 10

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

x_train = np.stack([x_train, x_train, x_train], axis=-1)
x_test = np.stack([x_test, x_test, x_test], axis=-1)

input_tensor = Input(shape=(img_height, img_width, 3))
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    x_train, y_train_categorical,
    epochs=10,
    batch_size=32,
    validation_data=(x_test, y_test_categorical)
)
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = y_test
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))
report = classification_report(y_true, y_pred_classes, output_dict=True)
accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1_score}")
model.save('resnet_mnist_model.h5')

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 28, 28, 3))
  warnings.warn(msg)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.6668 - loss: 1.2139

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 28, 28, 3))
  warnings.warn(msg)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 362s 186ms/step - accuracy: 0.6668 - loss: 1.2137 - val_accuracy: 0.8716 - val_loss: 0.4719
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 349s 186ms/step - accuracy: 0.8568 - loss: 0.4881 - val_accuracy: 0.8986 - val_loss: 0.3608
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 381s 186ms/step - accuracy: 0.8833 - loss: 0.3846 - val_accuracy: 0.9069 - val_loss: 0.3117
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 379s 184ms/step - accuracy: 0.8955 - loss: 0.3405 - val_accuracy: 0.9147 - val_loss: 0.2873
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 424s 206ms/step - accuracy: 0.9036 - loss: 0.3066 - val_accuracy: 0.9178 - val_loss: 0.2734
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 442s 206ms/step - accuracy: 0.9150 - loss: 0.2757 - val_accuracy: 0.9257 - val_loss: 0.2500
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 444s 207ms/step - accuracy: 0.9185 - loss: 0.2611 - val_accuracy: 0.9282 - val_loss: 0.2381
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 405s 187ms/step - accuracy: 0.9

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       980
           1       0.98      1.00      0.99      1135
           2       0.90      0.93      0.92      1032
           3       0.89      0.95      0.92      1010
           4       0.91      0.98      0.94       982
           5       0.92      0.89      0.91       892
           6       0.97      0.95      0.96       958
           7       0.95      0.94      0.94      1028
           8       0.89      0.89      0.89       974
           9       0.95      0.85      0.90      1009

    accuracy                           0.94     10000
   macro avg       0.94      0.93      0.93     10000
weighted avg       0.94      0.94      0.94     10000

Confusion Matrix:
[[ 939    1   11    1    0    4    9    1   13    1]
 [   0 1131    2    0    1    0    1    0    0    0]
 [   4    2  963   24    0    8    0   12   17    2]
 [   0    2   16  958    0   13    0    9